In [1]:
%load_ext autoreload
%autoreload 2

import sys, os, time, datetime

import psutil
process = psutil.Process(os.getpid())
print(f"Available memory: {int(psutil.virtual_memory().available / 1024 / 1024)} Mb")

# nbytes = process.memory_info().rss # bytes
# print(f"Process memory: {nbytes} bytes ({nbytes / 1024 / 1024} Mb)") 

import tensorflow as tf
from tensorflow import keras
print(f"Using Tensorflow v{tf.__version__}")

# For managing relative imports from notebook
if '..' not in sys.path: sys.path.append('..')

import config.config as dfc
import deepfake.dfutillib as df
import deepfake.modelutil as mutil

Available memory: 60089 Mb

Using Tensorflow v1.15.2


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# ------------------------- Model Instantiation -------------------------

lockdifflayers=True
mwrapper = mutil.ModelWrapper()
model = mwrapper.init_encdec_network(traindiff = not lockdifflayers)

if lockdifflayers:
    model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=1e-6), 
                  metrics=['binary_accuracy'], loss=tf.keras.losses.BinaryCrossentropy())
else:
    model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=1e-6), 
                  loss_weights=[1.0, 0.2], metrics=['accuracy', 'binary_accuracy'],
                  loss={'dflat_output': tf.keras.losses.MeanSquaredError(), 
                        'fake_output': tf.keras.losses.BinaryCrossentropy()})

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 360, 640, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 175, 158, 32) 11648       input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 87, 79, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 85, 77, 64)   18496       max_pooling2d[0][0]              
_____________

In [3]:
# ------------------------- Loaders Disk -> Model -------------------------
    
# Model data loaders
trloader = mutil.ModelLoader(split='train', traindiff=not lockdifflayers)
vdloader = mutil.ModelLoader(split='validate', traindiff=not lockdifflayers)
trdataset, vddataset = None, None # TF Dataset wrappers for data loaders

# Data shapes
inshape, labelshape = (None, 360, 640, 3), (None, 1)
imgoutshape = (None, dfc.TARGETSZ[0]*dfc.TARGETSZ[1]*3)

if lockdifflayers:
    trdataset = tf.data.Dataset.from_generator(generator=trloader.lazy_loader, 
        output_types=(tf.uint8, tf.float32), output_shapes=(inshape, labelshape))

    vddataset = tf.data.Dataset.from_generator(generator=vdloader.lazy_loader, 
        output_types=(tf.uint8, tf.float32), output_shapes=(inshape, labelshape))
    
else:
    trdataset = tf.data.Dataset.from_generator(generator=trloader.lazy_loader, 
        output_types=(tf.uint8, {'dflat_output': tf.uint8, 'fake_output': tf.uint8}),
        output_shapes=(inshape, {'dflat_output': imgoutshape, 'fake_output': labelshape}))

    vddataset = tf.data.Dataset.from_generator(generator=vdloader.lazy_loader, 
        output_types=(tf.uint8, {'dflat_output': tf.uint8, 'fake_output': tf.uint8}),
        output_shapes=(inshape, {'dflat_output': imgoutshape, 'fake_output': labelshape}))

# ------------------------- Callbacks Model -> Disk/Db -------------------------

# Timestamped logging/model directories
outdir = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tboarddir = f"{dfc.TBOARD_LOG}/{outdir}"
if not os.path.isdir(tboarddir): os.mkdir(tboarddir)

# Custom callback
epwrapdb_cb = mutil.PgdbWrapupCb(model)

# TF built-in callbacks
tboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=tboarddir, histogram_freq=1, write_graph=True, 
    write_images=False, update_freq=60, profile_batch=7)

savemodel_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=(f'{dfc.MODEL_STORE}/' + 'model.{epoch:02d}-{val_loss:.2f}.hdf5'), 
    monitor='val_loss', save_best_only=False, save_freq='epoch')


In [4]:
# Load last best model
model.load_weights(f'{dfc.MODEL_STORE}/20200330-001650/model.26-107.79.hdf5')

In [5]:
# ------------------------- Training -------------------------

history = model.fit(
    x=trdataset, steps_per_epoch=trloader.epochsz,
    validation_data=vddataset, validation_steps=vdloader.epochsz,
    callbacks=[tboard_cb, savemodel_cb, epwrapdb_cb],
    initial_epoch=0, verbose=2, epochs=1000)

Train on 360 steps, validate on 40 steps
Epoch 1/1000
MLoaderID: 139733648328912-train, loading blk_id: 179
5/360 batches trained. Time past: 0.52 min, ETA remain: 36.64 min, avg/batch: 6.19 sec
72/360 batches trained. Time past: 1.00 min, ETA remain: 4.01 min, avg/batch: 0.84 sec
144/360 batches trained. Time past: 1.73 min, ETA remain: 2.60 min, avg/batch: 0.72 sec
216/360 batches trained. Time past: 2.48 min, ETA remain: 1.65 min, avg/batch: 0.69 sec
288/360 batches trained. Time past: 3.28 min, ETA remain: 0.82 min, avg/batch: 0.68 sec
360/360 batches trained. Time past: 4.00 min, ETA remain: 0.00 min, avg/batch: 0.67 sec
MLoaderID: 139733648330480-validate, loading blk_id: 179
360/360 - 343s - loss: 125.4925 - binary_accuracy: 0.7884 - val_loss: 0.4886 - val_binary_accuracy: 0.8533
Epoch 2/1000
MLoaderID: 139733648328912-train, loading blk_id: 398
5/360 batches trained. Time past: 0.06 min, ETA remain: 4.59 min, avg/batch: 0.78 sec
72/360 batches trained. Time past: 0.82 min, ETA 

In [ ]:
datetime.datetime.now().strftime('%Y%m%d-%H%M%S')